In [1]:
import random
import itertools 
from poker import *

In [2]:
print("Standard 52-card deck for poker:\n")
print(deck)
print("\n5 possible hands of 5-card-stud:")
deal(numhands=5, n=5)

Standard 52-card deck for poker:

['2♥', '2♣', '2◆', '2♠', '3♥', '3♣', '3◆', '3♠', '4♥', '4♣', '4◆', '4♠', '5♥', '5♣', '5◆', '5♠', '6♥', '6♣', '6◆', '6♠', '7♥', '7♣', '7◆', '7♠', '8♥', '8♣', '8◆', '8♠', '9♥', '9♣', '9◆', '9♠', 'T♥', 'T♣', 'T◆', 'T♠', 'J♥', 'J♣', 'J◆', 'J♠', 'Q♥', 'Q♣', 'Q◆', 'Q♠', 'K♥', 'K♣', 'K◆', 'K♠', 'A♥', 'A♣', 'A◆', 'A♠']

5 possible hands of 5-card-stud:


[['2◆', '7♠', '5♣', 'T♣', '3♥'],
 ['8♠', '2♠', '8◆', '4♠', 'K◆'],
 ['Q♠', '3◆', '6♥', 'T♠', 'J◆'],
 ['7♣', '2♣', '5♥', 'A♣', 'Q◆'],
 ['4♣', 'A♥', '9◆', '6♣', '6◆']]

### How many possible combinations of 5 card hands exist in a 52 card deck?
- 13 unique ranks '2, 3, 4, 5, 6, 7, 8, 9, Ten, Jack, Queen, King, Ace'
- 4 unique suits '♥♣◆♠' Hearts, Clubs, Diamonds, Spades

In [3]:
# the quick answer
hands = [h for h in itertools.combinations(deck, 5)]
len(hands)

2598960

### Wow, Over 2 and a half million!  ---> 2,598,960 unique hands
- Let's unpack that

In [4]:
def n_choose_k(n, k):
    """
    Using combinatorics, we have `n` unique options
    that we can choose `k` cards from
    """
    return factorial(n)/(factorial(n-k)*factorial(k))

### Great but now we need to calculate `factorial`
- Quick refresher, 

$5!$ = $5*4*3*2*1$

- Yes, realize I can cheat and do

`from math import factorial`
- Rather, lets build it recursively 

In [5]:
def factorial(n):
    if n < 0:
        raise ValueError("Negative numbers have no factorial.")
    elif n <=1: 
        return 1
    else:
        return n * factorial(n-1)

fact_5 = factorial(5) 
assert fact_5 == 5*4*3*2*1
fact_5

120

### This works just fine, but there is a big drawback to using recursive functions, Stack Overflow
- No, not stackoverflow.com. But an actual [stack overflow](https://en.wikipedia.org/wiki/Stack_overflow)
### Instead, lets introduce  Dynamic Programming. 
- Storing previously computed values rather than recomputing them.
- In the case of `factorial` we build up the solution of a bigger problem from a smaller one.

In [6]:
def factorial_dp(n):
    fact = 1
    if n < 0:
        raise ValueError("Negative numbers have no factorial.")
    elif n <= 1:
        return fact
    for i in range(2, n+1):
        fact *=i
    return fact

factorial_dp(5)        

120

### Same idea with reduce 

In [7]:
from functools import reduce

In [8]:
def factorial_reduce(n):
    if n < 0:
        raise ValueError("Negative numbers have no factorial.")
    elif n <= 1:
        return 1
    return reduce(lambda x,y: x*y, range(2, n+1))
factorial_reduce(5)

120

`lambda x,y: x*y`

is equivalent to 

```def product(x, y):
    return x*y```

# Back to the problem at hand, `n_choose_k`
- Let's update the function to be able to choose a `factorial` function

In [9]:
def n_choose_k(n, k, func=factorial):
    """
    Using combinatorics, we have `n` unique options
    that we can choose `k` cards from
    and calculate with a factorial `func`
    """
    return func(n)//(func(n-k)*func(k)) # use floor division to remove floats

assert n_choose_k(52, 5, func=factorial) == 2598960 # testing functions
assert n_choose_k(52, 5, func=factorial_dp) == 2598960
assert n_choose_k(52, 5, func=factorial_reduce) == 2598960

In [10]:
%%timeit
n_choose_k(52, 5, func=factorial)

10000 loops, best of 3: 46.7 µs per loop


In [11]:
%%timeit
n_choose_k(52, 5, func=factorial_dp)

10000 loops, best of 3: 11.2 µs per loop


In [12]:
%%timeit
n_choose_k(52, 5, func=factorial_reduce)

10000 loops, best of 3: 31.9 µs per loop


### Dynamic programing is 3x faster than the recursion
- For completeness, 

In [13]:
from math import factorial

In [14]:
%%timeit
n_choose_k(52, 5, func=factorial)

The slowest run took 25.12 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.05 µs per loop


### So what is `n_choose_k` telling us?
- For starters, order doesn't matter.
- What if order mattered?
- Then we have a Permutation, i.e. an ordered Combination.

In [15]:
order_matters = reduce(lambda x,y: x*y, range(48, 53)) 
assert order_matters ==  52*51*50*49*48 
"If order mattered, we would have {} possible 5-card combinations from a 52-card deck".format(order_matters)

'If order mattered, we would have 311875200 possible 5-card combinations from a 52-card deck'

In [16]:
# as a function
def permuation_without_replacement(n, r):
    '''
    returns the number of `r` combinations w/out replacement
    from `n` options where order matter 
    '''
    return factorial(n)//factorial(n-r)

permuation_without_replacement(52, 5)

311875200

### Holy moly, over 311 million combinations!
- Yes, the order matters when we compare the ranks of hands, however,
- as can be seen below, when hands are first dealt, order doesn't matter.

In [17]:
for i in range(2):
    rand_hand = deal(numhands=1, n=5)[0] # random, assorted
    srtd_rand_hand = sorted(rand_hand) # sort ascending order
    srtd_rand_hand_rev = sorted(rand_hand, reverse=True) # sort descending order
    print("Three variations of the same hand: \n{} \n{} \n{}\n".format(rand_hand, 
                                                                       srtd_rand_hand, 
                                                                       srtd_rand_hand_rev))
    print("Hand ranks: \n{} \n{} \n{}\n".format(hand_rank(rand_hand),
                                                hand_rank(srtd_rand_hand),
                                                hand_rank(srtd_rand_hand_rev)))
    print("Assert all three hands have the same hand ranking:")
    print(hand_rank(rand_hand) == hand_rank(rand_hand) == hand_rank(rand_hand))
    print("-"*50)

Three variations of the same hand: 
['4♠', '5♣', '7♠', '6♠', 'K♣'] 
['4♠', '5♣', '6♠', '7♠', 'K♣'] 
['K♣', '7♠', '6♠', '5♣', '4♠']

Hand ranks: 
(0, [13, 7, 6, 5, 4]) 
(0, [13, 7, 6, 5, 4]) 
(0, [13, 7, 6, 5, 4])

Assert all three hands have the same hand ranking:
True
--------------------------------------------------
Three variations of the same hand: 
['A♠', '7♠', '6♣', 'T♥', '5♠'] 
['5♠', '6♣', '7♠', 'A♠', 'T♥'] 
['T♥', 'A♠', '7♠', '6♣', '5♠']

Hand ranks: 
(0, [14, 10, 7, 6, 5]) 
(0, [14, 10, 7, 6, 5]) 
(0, [14, 10, 7, 6, 5])

Assert all three hands have the same hand ranking:
True
--------------------------------------------------


### So, how do we remove the ordered condition?
First lets compare the 2 formulas:
     
`permuation_without_replacement` $ -- >\frac{n!}{(n − r)!}$

`n_choose_k` $ --> \frac{n!}{(n − k)!k!}$ 

Nearly identical, besides an additional $k!$ included in the `n_choose_k` denominator. 

For lack of better words, this helps us 'normalize' the order condition.

In [18]:
permuation_without_replacement(52, 5)//factorial(5)

2598960

In [19]:
n_choose_k(52, 5)

2598960

In [20]:
# TODO Dynamic Programming
# Knapsack Problem

___